# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [23]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [24]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [25]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'epa', 'fruits_db', 'local', 'met', 'petsitly_marketing', 'travel_db', 'uk_food']


In [26]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [27]:
# review the collections in our new database
collections = db.list_collection_names()
collections

['establishments']

In [41]:
# assign the collection to a variable
establishments = db['establishments']

In [42]:
# review a document in the establishments collection
document = establishments.find_one()
document

{'_id': ObjectId('680d4255c419eb47715d2333'),
 'FHRSID': 632212,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000043474',
 'BusinessName': 'Lenox House',
 'BusinessType': 'Hotel/bed & breakfast/guest house',
 'BusinessTypeID': 7842,
 'AddressLine1': '27 Granville Road',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT15 6DS',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2020-03-17T00:00:00',
 'LocalAuthorityCode': '182',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'scores': {'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.385585', 'latitude': '51.152411'},
 'RightToReply': '',
 'Distance': 4587.440906956158,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  'itemCount':

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [61]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}


In [62]:
# Insert the new restaurant into the collection
insert_result = establishments.insert_one(new_restaurant)

In [63]:
# Check that the new restaurant was inserted
if insert_result.inserted_id:
    print(f"New restaurant added with ID: {insert_result.inserted_id}")
else:
    print("Failed to add the new restaurant.")

New restaurant added with ID: 680d4c7eafb86fd205b0f563


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [64]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
result = establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"BusinessTypeID":1,"BusinessType":1, "_id":0}
)
pprint(result)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [65]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one(
    {"BusinessName": "Penang Flavours"},
    {"$set": {"BusinessTypeID": 1}}
)

UpdateResult({'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [66]:
# Confirm that the new restaurant was updated
updated_restaurant = establishments.find_one({"BusinessName": "Penang Flavours"})
pprint(updated_restaurant)

{'AddressLine3': 'Greenwich',
 'BusinessName': 'Penang Flavours',
 'BusinessTypeID': 1,
 'RatingValue': None,
 '_id': ObjectId('680d4454afb86fd205b0f560')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [67]:
# Find how many documents have LocalAuthorityName as "Dover"
count_dover = establishments.count_documents({"LocalAuthorityName": "Dover"})
print("Documents with Dover:", count_dover)

Documents with Dover: 0


In [68]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({"LocalAuthorityName": "Dover"})

DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

In [69]:
# Check if any remaining documents include Dover
count_dover = establishments.count_documents({"LocalAuthorityName": "Dover"})
print("Documents with Dover:", count_dover)

Documents with Dover: 0


In [70]:
# Check that other documents remain with 'find_one'
sample_non_dover = establishments.find_one({"LocalAuthorityName": {"$ne": "Dover"}})
pprint(sample_non_dover)

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('680d4255c419eb47715d2615'),
 'geocode': {'latitude': 51.086058, 'longitude': 1.196408},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extrac

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [71]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many(
    {"geocode.latitude": {"$type": "string"}},
    [{"$set": {"geocode.latitude": {"$toDouble": "$geocode.latitude"}}}]
)

establishments.update_many(
    {"geocode.longitude": {"$type": "string"}},
    [{"$set": {"geocode.longitude": {"$toDouble": "$geocode.longitude"}}}]
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [72]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt","None"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [73]:
# Change the data type from String to Integer for RatingValue
establishments.update_many(
    {"RatingValue": {"$type": "string"}},
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]
)

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [74]:
# Check that the coordinates and rating value are now numbers
doc = establishments.find_one({})
print(type(doc["geocode"]["latitude"]))
print(type(doc["geocode"]["longitude"]))
print(type(doc["RatingValue"]))


<class 'float'>
<class 'float'>
<class 'int'>


In [ ]:
#I ran the whole code a few times and created multiple records for Penang Flavours. Here's how I check for duplicates:
results = list(establishments.find({"BusinessName": "Penang Flavours"}))
print(f"Found {len(results)} Penang Flavours records.")
for doc in results:
    pprint(doc)

Found 3 Penang Flavours records.
{'AddressLine3': 'Greenwich',
 'BusinessName': 'Penang Flavours',
 'BusinessTypeID': 1,
 'RatingValue': None,
 '_id': ObjectId('680d4454afb86fd205b0f560')}
{'AddressLine3': 'Greenwich',
 'BusinessName': 'Penang Flavours',
 'RatingValue': None,
 '_id': ObjectId('680d47fdafb86fd205b0f562')}
{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('680d4c7eafb86fd205b0f563'),
 'geocode': {'latitude': 51.490142, 'longitude': 0.08384},
 'scores': {'C

In [77]:
from bson.objectid import ObjectId
#Delete the two incomplete records
ids_to_delete = [
    ObjectId('680d4454afb86fd205b0f560'),
    ObjectId('680d47fdafb86fd205b0f562')
]

delete_result = establishments.delete_many({'_id': {'$in': ids_to_delete}})
print(f"Deleted {delete_result.deleted_count} records.")

Deleted 2 records.


In [78]:
#Re-check
results = list(establishments.find({"BusinessName": "Penang Flavours"}))
print(f"Found {len(results)} Penang Flavours records.")
for doc in results:
    pprint(doc)

Found 1 Penang Flavours records.
{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('680d4c7eafb86fd205b0f563'),
 'geocode': {'latitude': 51.490142, 'longitude': 0.08384},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


In [ ]:
#Yay!